In [1]:
import time
import torch
from torchvision import transforms as T
from tqdm import tqdm
from tempo3.data.datasets import video_dataset, video_dataset_h5
from tempo3.data.pdfs import pdf_index
from tqdm import tqdm

import os
import h5py
from PIL import Image
import matplotlib.pyplot as plt

/home/yasin/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Create folder structure
root = "./datasets/"
uav_root = os.path.join(root, "UAV123")
uav_h5_root = os.path.join(root, "uav123")

if "uav123" not in os.listdir(root):
    os.makedirs(uav_h5_root)

In [3]:
frames = h5py.File(os.path.join(uav_h5_root, "frames.hdf5"), "w")

In [4]:
frame_shape = torch.tensor(T.ToTensor()(Image.open(os.path.join(uav_root, "bike1", "000001.jpg")))).shape
frame_shape

/tmp/ipykernel_5593/2658190090.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frame_shape = torch.tensor(T.ToTensor()(Image.open(os.path.join(uav_root, "bike1", "000001.jpg")))).shape


torch.Size([3, 720, 1280])

In [5]:
for v in os.listdir(uav_root)[:1]:
    frames_root = os.path.join(uav_root, v)
    num_images = len(os.listdir(frames_root))

    frame_paths = [os.path.join(frames_root, f) for f in sorted(os.listdir(frames_root))]
    # data = torch.cat([T.ToTensor()(Image.open(f))[None] for f in frame_paths])

    dset = frames.create_dataset(v, (num_images, *frame_shape), dtype='f')
    
    tf = T.ToTensor()
    for i, p in enumerate(frame_paths):
        dset[i] = tf(Image.open(p))

In [9]:
frames["person18"]

<HDF5 dataset "person18": shape (1393, 3, 720, 1280), type "<f4">

In [2]:
names = sorted(list(os.listdir("datasets/ASL-big/frames/")))
images = [Image.open(f"datasets/ASL-big/frames/{name}", mode="r")for name in names]

In [3]:
transform = T.Compose([
    T.Resize(128),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [4]:
images_t = [transform(i) for i in images]
images_t = torch.stack(images_t)
images_t.shape

torch.Size([3757, 3, 128, 128])

In [5]:
split = int(0.8*len(images_t))
images_t_train = images_t[:split]
images_t_test = images_t[split:]

In [6]:
images_t_train.shape, images_t_test.shape

(torch.Size([3005, 3, 128, 128]), torch.Size([752, 3, 128, 128]))

In [7]:
f_train = h5py.File("frames_train.hdf5", "w")
f_test = h5py.File("frames_test.hdf5", "w")

In [8]:
dset_train = f_train.create_dataset("frames", tuple(images_t_train.shape), data=images_t_train, dtype='f')
dset_test = f_test.create_dataset("frames", tuple(images_t_test.shape), data=images_t_test, dtype='f')

In [10]:
dset_train.shape, dset_test.shape

((3005, 3, 128, 128), (752, 3, 128, 128))

In [11]:
f_train.close(), f_test.close()

(None, None)